In [35]:
#r "nuget:Microsoft.ML,1.5.0-preview"
#r "nuget:Microsoft.ML.AutoML,0.17.0-preview"
#r "nuget:Microsoft.Data.Analysis,0.1.0"
#r "nuget:Microsoft.ML.OnnxConverter,0.17.0-preview"
#r "nuget:Microsoft.ML.OnnxRuntime,1.1.0"

Installing package Microsoft.ML, version 1.5.0-preview....done!

Successfully added reference to package Microsoft.ML, version 1.5.0-preview

Installing package Microsoft.ML.AutoML, version 0.17.0-preview....done!

Successfully added reference to package Microsoft.ML.AutoML, version 0.17.0-preview

Installing package Microsoft.Data.Analysis, version 0.1.0....done!

Successfully added reference to package Microsoft.Data.Analysis, version 0.1.0

Installing package Microsoft.ML.OnnxConverter, version 0.17.0-preview....done!

Successfully added reference to package Microsoft.ML.OnnxConverter, version 0.17.0-preview

Installing package Microsoft.ML.OnnxRuntime, version 1.1.0...............done!

Successfully added reference to package Microsoft.ML.OnnxRuntime, version 1.1.0

In [36]:
using Microsoft.Data.Analysis;
using XPlot.Plotly;

In [37]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.RowCount); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [38]:
using System.IO;
using System.Net.Http;
string ordersPath = "order-details-fullprice.csv";
if (!File.Exists(ordersPath))
{
    var contents = new HttpClient()
        .GetStringAsync("https://raw.githubusercontent.com/rondagdag/onnx-pected/master/GenerateONNX-AutoML-Orders/GenerateONNX-AutoML/Data/order-details-fullprice.csv").Result;
        
    File.WriteAllText("order-details-fullprice.csv", contents);
}

In [39]:
var ordersData = DataFrame.LoadCsv(ordersPath);
ordersData

0,11,14,12,True
1,42,9.8,10,True
2,72,34.8,5,True
3,14,18.6,9,True
4,51,42.4,40,True
5,41,7.7,10,True
6,51,42.4,35,False
7,65,16.8,15,False
8,22,16.8,6,False
9,57,15.6,15,False
10,65,16.8,20,True


In [40]:
Chart.Plot(
    new Graph.Histogram()
    {
        x = ordersData["PayFullPrice"],
        nbinsx = 20
    }
)

In [41]:
var chart = Chart.Plot(
    new Graph.Scattergl()
    {
        x = ordersData["UnitPrice"],
        y = ordersData["Quantity"],
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = ordersData["PayFullPrice"],
            colorscale = "Jet"
        }
    }
);

chart.Width = 600;
chart.Height = 600;
display(chart);

In [42]:
static T[] Shuffle<T>(T[] array)
{
    Random rand = new Random();
    for (int i = 0; i < array.Length; i++)
    {
        int r = i + rand.Next(array.Length - i);
        T temp = array[r];
        array[r] = array[i];
        array[i] = temp;
    }
    return array;
}

int[] randomIndices = Shuffle(Enumerable.Range(0, (int)ordersData.RowCount).ToArray());
int testSize = (int)(ordersData.RowCount * .1);
int[] trainRows = randomIndices[testSize..];
int[] testRows = randomIndices[..testSize];

DataFrame orders_train = ordersData[trainRows];
DataFrame orders_test = ordersData[testRows];

display(orders_train.RowCount);
display(orders_test.RowCount);

1940

215

In [43]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

In [44]:
var mlContext = new MLContext();

In [45]:
%%time

var experiment = mlContext.Auto().CreateBinaryClassificationExperiment(maxExperimentTimeInSeconds: 30);
var result = experiment.Execute(orders_train, labelColumnName:"PayFullPrice");

Wall time: 31916.882100000003ms

In [50]:
var scatters = result.RunDetails.Where(d => d.ValidationMetrics != null).GroupBy(    
    r => r.TrainerName,
    (name, details) => new Graph.Scattergl()
    {
        name = name,
        x = details.Select(r => r.RuntimeInSeconds),
        y = details.Select(r => r.ValidationMetrics.Accuracy),
        mode = "markers",
        marker = new Graph.Marker() { size = 12 }
    });

var chart = Chart.Plot(scatters);
chart.WithXTitle("Training Time");
chart.WithYTitle("Error");
display(chart);

Console.WriteLine($"Best Trainer:{result.BestRun.TrainerName}");

Best Trainer:SdcaLogisticRegressionBinary


In [51]:
RunDetail<BinaryClassificationMetrics> best = result.BestRun;
ITransformer trainedModel = best.Model;

In [52]:
var testResults = result.BestRun.Model.Transform(orders_test);

var metrics = mlContext.BinaryClassification.Evaluate(testResults, labelColumnName: "PayFullPrice", scoreColumnName: "Score");


In [75]:
Console.WriteLine();
Console.WriteLine("Model quality metrics evaluation");
Console.WriteLine("--------------------------------");
Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");
Console.WriteLine($"Auc: {metrics.AreaUnderRocCurve:P2}");
Console.WriteLine($"F1Score: {metrics.F1Score:P2}");
Console.WriteLine("=============== End of model evaluation ===============");


Model quality metrics evaluation
--------------------------------
Accuracy: 64.19%
Auc: 58.55%
F1Score: 76.45%
=============== End of model evaluation ===============


In [76]:
var MODEL_NAME = "orders.onnx";
using (var stream = File.Create(MODEL_NAME))
{
    mlContext.Model.ConvertToOnnx(trainedModel, ordersData, stream);
}
Console.WriteLine("The model is saved to {0}", MODEL_NAME);

The model is saved to orders.onnx


In [77]:
using Microsoft.ML.OnnxRuntime;
using Microsoft.ML.OnnxRuntime.Tensors;

In [78]:
private static NamedOnnxValue GetNamedOnnxValue<T>(IReadOnlyDictionary<string, NodeMetadata> inputMeta, string column, T value)
{
    T[] inputDataInt = new T[] { value };
    var tensor = new DenseTensor<T>(inputDataInt, inputMeta[column].Dimensions);
    var namedOnnxValue = NamedOnnxValue.CreateFromTensor<T>(column, tensor);
    return namedOnnxValue;
}

In [86]:
var session = new InferenceSession(MODEL_NAME);

/*cont onnx*/

var inputMeta = session.InputMetadata;

var container = new List<NamedOnnxValue>();
//10298,36,15.20,40,0.25
container.Add(GetNamedOnnxValue<float>(inputMeta, "ProductID", 41));
container.Add(GetNamedOnnxValue<float>(inputMeta, "UnitPrice", 7.7f));
container.Add(GetNamedOnnxValue<float>(inputMeta, "Quantity", 10f));
container.Add(GetNamedOnnxValue<bool>(inputMeta, "PayFullPrice", false));

In [87]:
var result = session.Run(container);
var output = result.First(x => x.Name == "PredictedLabel0").AsTensor<bool>().GetValue(0);

Console.WriteLine($"**********************************************************************");
Console.WriteLine($"Predicted Pay Full Price: {output:0.####}, actual : true");
Console.WriteLine($"**********************************************************************");

**********************************************************************
Predicted Pay Full Price: True, actual : true
**********************************************************************
